In [ ]:
!pip install langchain
!pip install openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain import FewShotPromptTemplate
from langchain.schema import(AIMessage,HumanMessage,SystemMessage)
import os

In [2]:
os.environ['OPENAI_API_KEY'] = 'sk-xxxx'

In [3]:
def few_shot_cot(claim_text):
  examples = [
  {"claim": """ # The claim is that In 1959 , former Chilean boxer Alfredo Cornejo Cuevas (born June 6, 1933) won the gold medal in the welterweight division at the Pan American Games (held in Chicago, United States, from August 27 to September 7) in Chicago, United States, and the world amateur welterweight title in Mexico City. """,
  "program": """
  def program ():
    fact_1 = Verify (" Alfredo Cornejo Cuevas was born in June 6 , 1933. ")
    fact_2 = Verify (" Alfredo Cornejo Cuevas won the gold medal in the welterweight division at the Pan American Games in 1959. ")
    fact_3 = Verify (" The Pan American Games in 1959 was held in Chicago , United States , from August 27 to September 7.")
    fact_4 = Verify (" Alfredo Cornejo Cuevas won the world amateur welterweight title in Mexico City .")
    label = Predict ( fact_1 and fact_2 and fact_3 and fact_4 )
  """},

  {"claim": """ # The claim is that The Footwork FA12 , which was intended to start the season , finally debuted at the San Marino Grand Prix , a Formula One motor race held at Imola on 28 April 1991.""",
  "program": """
  def program ():
    fact_1 = Verify (" The Footwork FA12 , which was intended to start the season .")
    fact_2 = Verify (" The Footwork FA12 finally debuted at the San Marino Grand Prix .")
    fact_3 = Verify (" The San Marino Grand Prix was a Formula One motor race held at Imola on 28 April 1991. ")
    label = Predict ( fact_1 and fact_2 and fact_3 )
  """}]

  example_template = """claim: {claim}
program: {program}"""
  prefix = """Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . """
  suffix ="""claim: {claim}"""
  example_prompt = PromptTemplate(input_variables = ["claim","program"],template=example_template)

  few_shot_prompt_template = FewShotPromptTemplate(
      examples = examples,
      example_prompt = example_prompt,
      prefix = prefix,
      suffix = suffix,
      input_variables = ["claim"],
      example_separator = "\n\n"
  )

  max_tokens_chat = 2000

  llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo',max_tokens=max_tokens_chat)
  from langchain.prompts.chat import (
      ChatPromptTemplate,
      SystemMessagePromptTemplate,
      HumanMessagePromptTemplate,
  )
  template = ''
  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template="{text}"
  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
  chain = LLMChain(llm=llm, prompt=chat_prompt)
  answer = chain.run(text=few_shot_prompt_template.format(claim=claim_text))

  return answer,few_shot_prompt_template.format(claim=claim_text)

In [4]:
answer,input = few_shot_cot('# The claim is that Sumo wrestler Toyozakura Toshiaki committed match -fixing , ending his career in 2011 that started in 1989.')

In [5]:
print(input)
print(' ')
print(answer)

Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . 

claim:  # The claim is that In 1959 , former Chilean boxer Alfredo Cornejo Cuevas (born June 6, 1933) won the gold medal in the welterweight division at the Pan American Games (held in Chicago, United States, from August 27 to September 7) in Chicago, United States, and the world amateur welterweight title in Mexico City. 
program: 
  def program ():
    fact_1 = Verify (" Alfredo Cornejo Cuevas was born in June 6 , 1933. ")
    fact_2 = Verify (" Alfredo Cornejo Cuevas won the gold medal in the welterweight division at the Pan American Games in 1959. ")
    fact_3 = Verify (" The Pan American Games in 1959 was held in Chicago , United States , from August 27 to September 7.")
    fact_4 = Verify (" Alfredo Co